# PSyKE's demo for regression tasks

Some imports.

In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from os.path import exists
from psyke import Extractor
from psyke.utils.logic import pretty_theory
from psyke.regression import FeatureRanker
from psyke.optimizer.pedro import PEDRO
from sklearn.neighbors import KNeighborsRegressor
from psyke.optimizer import Objective
from psyke.optimizer.crash import CRASH

import warnings
warnings.simplefilter("ignore")

In [2]:
algorithms = ["contingency", "contingency", "anticipate", "anticipate", "contingency", "anticipate"]
models = [
    "CONTINGENCY_no_input-memory_DecisionTree_MaxDepth10",
    "CONTINGENCY_no_input-time_DecisionTree_MaxDepth10",
    "ANTICIPATE_no_input-memory_DecisionTree_MaxDepth10",
    "ANTICIPATE_no_input-time_DecisionTree_MaxDepth10",
    "CONTINGENCY_input-cost_DecisionTree_MaxDepth15",
    "ANTICIPATE_input-cost_DecisionTree_MaxDepth15"
]
models = [
    pickle.load(open("test/resources/datasets/models/" + algorithm + "/" + path, 'rb'))
    for path, algorithm in zip(models, algorithms)
]

In [3]:
def process(algorithm):
    df = pd.read_csv("test/resources/datasets/datasets/" + algorithm + "_trainDataset.csv")

    # Removes header entries
    df = df[df['sol(keuro)'] != 'sol(keuro)']

    # Fixed stuff which is always there
    df['PV(kW)'] = df['PV(kW)'].map(lambda entry: entry[1:-1].split())
    df['PV(kW)'] = df['PV(kW)'].map(lambda entry: list(np.float_(entry)))
    df['Load(kW)'] = df['Load(kW)'].map(lambda entry: entry[1:-1].split())
    df['Load(kW)'] = df['Load(kW)'].map(lambda entry: list(np.float_(entry)))

    X = pd.DataFrame()

    X['PV_mean'] = df['PV(kW)'].map(lambda entry: np.array(entry).mean())
    X['PV_std'] = df['PV(kW)'].map(lambda entry: np.array(entry).std())
    X['Load_mean'] = df['Load(kW)'].map(lambda entry: np.array(entry).mean())
    X['Load_std'] = df['Load(kW)'].map(lambda entry: np.array(entry).std())
    X['nScenarios'] = df['nScenarios']
    X['cost'] = df['sol(keuro)']
    X['time'] = df['time(sec)']
    X['memo'] = df['memAvg(MB)']

    X.to_csv("test/resources/datasets/datasets/" + algorithm + ".csv", index = False)

    return X

In [4]:
toRemove = [
    ['PV_mean', 'PV_std', 'Load_mean', 'Load_std', 'time', 'cost'],
    ['PV_mean', 'PV_std', 'Load_mean', 'Load_std', 'memo', 'cost'],
    ['PV_mean', 'PV_std', 'Load_mean', 'Load_std', 'time', 'cost'],
    ['PV_mean', 'PV_std', 'Load_mean', 'Load_std', 'memo', 'cost'],
    ["time", "memo"],
    ["time", "memo"]
]

features = [
    ["nTraces"],
    ["nTraces"],
    ["nScenarios"],
    ["nScenarios"],
    ['PV_mean', 'PV_std', 'Load_mean', 'Load_std', 'nTraces'],
    ['PV_mean', 'PV_std', 'Load_mean', 'Load_std', 'nScenarios']
]

targets = ["memo", "time", "memo", "time", "cost", "cost"]

output = []

In [5]:
from psyke.regression import Grid, FixedStrategy

i = 0
for rem, feat, target, algorithm, model in zip(toRemove, features, targets, algorithms, models):
    if i == 0:
        i += 1
        continue

    print(algorithm, target, len(feat))
    name = f"test/resources/datasets/datasets/{algorithm}.csv"

    if not exists(name):
        process(algorithm)

    dataset = pd.read_csv(name)

    dataset = dataset.drop(rem, axis = 1)

    train, test = train_test_split(dataset, test_size=0.2, random_state=10)
    model.fit(train.iloc[:, :-1], train.iloc[:, -1])
    E = abs(model.predict(test.iloc[:, :-1]) - test.iloc[:, -1])
    print("MAE = {:.2f}".format(E.mean()))
    output.append(("model", algorithm, target, E.mean()))
    #plt.plot(E, ".")
    #plt.show()

    #pedro = PEDRO(model, train, max_mae_increase=1.2, min_rule_decrease=0.9, readability_tradeoff=0.1,
    #          max_depth=5, patience=1, algorithm=PEDRO.Algorithm.GRIDREX, objective=Objective.MODEL)
    #pedro.search()
    #best = pedro.get_best()[0]

    ranked = FeatureRanker(dataset.columns[:-1]).fit(model, dataset.iloc[:, :-1]).rankings()
    #gridREx = Extractor.gridrex(model, best[3], threshold=best[2])
    gridREx = Extractor.gridrex(model, Grid(2, FixedStrategy(2)), threshold=0.75)
    theory_from_gridREx = gridREx.extract(train)
    print('GridREx performance ({} rules):\nMAE = {:.2f}\nMAE fidelity = {:.2f}\n'
          .format(gridREx.n_rules, gridREx.mae(test), gridREx.mae(test, model)))
    print('GridREx extracted rules:\n\n' + pretty_theory(theory_from_gridREx))
    break

contingency time 1
MAE = 0.84
GridREx performance (3 rules):
MAE = 1.00
MAE fidelity = 0.75

GridREx extracted rules:

time(NTraces, Time) :-
    NTraces in [0.99, 50.5], Time is 4.69 + 0.63 * NTraces.
time(NTraces, Time) :-
    NTraces in [50.5, 75.25], Time is 2.88 + 0.63 * NTraces.
time(NTraces, Time) :-
    NTraces in [75.25, 100.00], Time is 8.31 + 0.55 * NTraces.


In [5]:
for rem, feat, target, algorithm, model in zip(toRemove, features, targets, algorithms, models):

    print(algorithm, target, len(feat))
    name = f"test/resources/datasets/datasets/{algorithm}.csv"

    if not exists(name):
        process(algorithm)

    dataset = pd.read_csv(name)

    dataset = dataset.drop(rem, axis = 1)

    train, test = train_test_split(dataset, test_size=0.2, random_state=10)
    model.fit(train.iloc[:, :-1], train.iloc[:, -1])
    E = abs(model.predict(test.iloc[:, :-1]) - test.iloc[:, -1])
    print("MAE = {:.2f}".format(E.mean()))
    output.append(("model", algorithm, target, E.mean()))
    #plt.plot(E, ".")
    #plt.show()

    pedro = PEDRO(model, train, max_mae_increase=1.2, min_rule_decrease=0.9, readability_tradeoff=0.1,
              max_depth=5, patience=1, algorithm=PEDRO.Algorithm.GRIDREX, objective=Objective.MODEL)
    pedro.search()
    best = pedro.get_best()[0]

    ranked = FeatureRanker(dataset.columns[:-1]).fit(model, dataset.iloc[:, :-1]).rankings()
    gridREx = Extractor.gridrex(model, best[3], threshold=best[2])
    theory_from_gridREx = gridREx.extract(train)
    print('GridREx performance ({} rules):\nMAE = {:.2f}\nMAE fidelity = {:.2f}\n'
          .format(gridREx.n_rules, gridREx.mae(test), gridREx.mae(test, model)))
    print('GridREx extracted rules:\n\n' + pretty_theory(theory_from_gridREx))

    crash = CRASH(model, train, readability_tradeoff=0.1, max_depth=5, patience=1,
                  algorithm=CRASH.Algorithm.CReEPy, objective=Objective.MODEL)
    crash.search()
    best =  crash.get_best()[0]

    creepy = Extractor.creepy(model, depth=best[2], error_threshold=best[3], constant=False)
    theory_from_creepy = creepy.extract(train)
    print('CReEPy performance ({} rules):\nMAE = {:.2f}\nMAE fidelity = {:.2f}\n'
          .format(creepy.n_rules, creepy.mae(test), creepy.mae(test, model)))

    crash = CRASH(model, train, readability_tradeoff=0.1, max_depth=5, patience=1,
                  algorithm=CRASH.Algorithm.CREAM, objective=Objective.MODEL)
    crash.search()
    best =  crash.get_best()[0]

    cream = Extractor.cream(model, depth=best[2], error_threshold=best[3], constant=False)
    theory_from_cream = cream.extract(train)
    print('CREAM performance ({} rules):\nMAE = {:.2f}\nMAE fidelity = {:.2f}\n'
          .format(cream.n_rules, cream.mae(test), cream.mae(test, model)))

    print()
    print()
    print()
    print()

contingency memo 1
MAE = 3.64
Algorithm.CReEPy. Depth: 1. Threshold = 3.43. 

KeyboardInterrupt: 

In [7]:
for rem, feat, target, algorithm in zip(toRemove, features, targets, algorithms):

    print(algorithm, target, len(feat))
    name = f"test/resources/datasets/datasets/{algorithm}.csv"

    if not exists(name):
        process(algorithm)

    dataset = pd.read_csv(name)

    dataset = dataset.drop(rem, axis = 1)

    train, test = train_test_split(dataset, test_size=0.2, random_state=10)
    model = KNeighborsRegressor(n_neighbors=1).fit(train.iloc[:, :-1], train.iloc[:, -1])
    E = abs(model.predict(test.iloc[:, :-1]) - test.iloc[:, -1])
    print("MAE = {:.2f}".format(E.mean()))
    output.append(("model", algorithm, target, E.mean()))

    crash = CRASH(model, train, readability_tradeoff=0.1, max_depth=5, patience=1,
                  algorithm=CRASH.Algorithm.CReEPy, objective=Objective.DATA)
    crash.search()
    best =  crash.get_best()[0]

    creepy = Extractor.creepy(model, depth=best[2], error_threshold=best[3], constant=False)
    theory_from_creepy = creepy.extract(train)
    print('CReEPy performance ({} rules):\nMAE = {:.2f}\nMAE fidelity = {:.2f}\n'
          .format(creepy.n_rules, creepy.mae(test), creepy.mae(test, model)))

    crash = CRASH(model, train, readability_tradeoff=0.1, max_depth=5, patience=1,
                  algorithm=CRASH.Algorithm.CREAM, objective=Objective.DATA)
    crash.search()
    best =  crash.get_best()[0]

    cream = Extractor.cream(model, depth=best[2], error_threshold=best[3], constant=False)
    theory_from_cream = cream.extract(train)
    print('CREAM performance ({} rules):\nMAE = {:.2f}\nMAE fidelity = {:.2f}\n'
          .format(cream.n_rules, cream.mae(test), cream.mae(test, model)))

    print()
    print()
    print()
    print()

contingency memo 1
MAE = 2.82
Algorithm.CReEPy. Depth: 1. Threshold = 2.69. MAE = 4.75, 2 rules
Algorithm.CReEPy. Depth: 1. Threshold = 4.18. MAE = 4.83, 2 rules

Algorithm.CReEPy. Depth: 2. Threshold = 2.69. MAE = 4.83, 2 rules
Algorithm.CReEPy. Depth: 2. Threshold = 4.18. MAE = 4.79, 2 rules
Algorithm.CReEPy. Depth: 2. Threshold = 5.68. MAE = 4.75, 2 rules
Algorithm.CReEPy. Depth: 2. Threshold = 7.17. MAE = 4.75, 2 rules

Algorithm.CReEPy. Depth: 3. Threshold = 2.69. MAE = 4.81, 3 rules
Algorithm.CReEPy. Depth: 3. Threshold = 4.18. MAE = 4.79, 2 rules
Algorithm.CReEPy. Depth: 3. Threshold = 5.68. MAE = 4.81, 2 rules

Algorithm.CReEPy. Depth: 4. Threshold = 2.69. MAE = 4.76, 3 rules
Algorithm.CReEPy. Depth: 4. Threshold = 4.18. MAE = 4.81, 2 rules
Algorithm.CReEPy. Depth: 4. Threshold = 5.68. MAE = 4.79, 3 rules

**********************
*Best Algorithm.CReEPy*
**********************
MAE = 4.75, 2 rules
Threshold = 2.69
Depth = 1

**********************
*Best   MAE  *
******************

In [9]:
for i, (rem, feat, target, algorithm, model) in enumerate(zip(toRemove, features, targets, algorithms, models)):

    print(algorithm, target, len(feat))

    if i < 4:
        print("skip")
        continue

    name = f"test/resources/datasets/datasets/{algorithm}.csv"

    if not exists(name):
        process(algorithm)

    dataset = pd.read_csv(name)

    dataset = dataset.drop(rem, axis = 1)

    train, test = train_test_split(dataset, test_size=0.2, random_state=10)
    model.fit(train.iloc[:, :-1], train.iloc[:, -1])
    E = abs(model.predict(test.iloc[:, :-1]) - test.iloc[:, -1])
    print("MAE = {:.2f}".format(E.mean()))
    output.append(("model", algorithm, target, E.mean()))

    crash = CRASH(model, train, readability_tradeoff=0.5, max_depth=5, patience=2,
                  algorithm=CRASH.Algorithm.CReEPy, objective=Objective.MODEL)
    crash.search()
    best =  crash.get_best()[0]

    creepy = Extractor.creepy(model, depth=best[2], error_threshold=best[3], constant=False)
    theory_from_creepy = creepy.extract(train)
    print('CReEPy performance ({} rules):\nMAE = {:.2f}\nMAE fidelity = {:.2f}\n'
          .format(creepy.n_rules, creepy.mae(test), creepy.mae(test, model)))

    print()

    model = KNeighborsRegressor(n_neighbors=1).fit(train.iloc[:, :-1], train.iloc[:, -1])
    crash = CRASH(model, train, readability_tradeoff=0.5, max_depth=5, patience=2,
                  algorithm=CRASH.Algorithm.CReEPy, objective=Objective.DATA)
    crash.search()
    best =  crash.get_best()[0]

    creepy = Extractor.creepy(model, depth=best[2], error_threshold=best[3], constant=False)
    theory_from_creepy = creepy.extract(train)
    print('CReEPy performance ({} rules):\nMAE = {:.2f}\nMAE fidelity = {:.2f}\n'
          .format(creepy.n_rules, creepy.mae(test), creepy.mae(test, model)))

    print()

contingency memo 1
skip
contingency time 1
skip
anticipate memo 1
skip
anticipate time 1
skip
contingency cost 5
MAE = 1.50
Algorithm.CReEPy. Depth: 1. Threshold = 0.50. MAE = 25.63, 2 rules
Algorithm.CReEPy. Depth: 1. Threshold = 0.78. MAE = 27.54, 2 rules
Algorithm.CReEPy. Depth: 1. Threshold = 27.54. MAE = 27.67, 2 rules

Algorithm.CReEPy. Depth: 2. Threshold = 0.50. MAE = 26.13, 3 rules
Algorithm.CReEPy. Depth: 2. Threshold = 0.78. MAE = 26.24, 3 rules
Algorithm.CReEPy. Depth: 2. Threshold = 26.24. MAE = 23.74, 3 rules
Algorithm.CReEPy. Depth: 2. Threshold = 51.69. MAE = 27.39, 2 rules
Algorithm.CReEPy. Depth: 2. Threshold = 77.14. MAE = 28.03, 2 rules

Algorithm.CReEPy. Depth: 3. Threshold = 0.50. MAE = 24.22, 4 rules
Algorithm.CReEPy. Depth: 3. Threshold = 0.78. MAE = 23.26, 4 rules
Algorithm.CReEPy. Depth: 3. Threshold = 1.06. MAE = 23.67, 4 rules
Algorithm.CReEPy. Depth: 3. Threshold = 23.67. MAE = 25.08, 4 rules

Algorithm.CReEPy. Depth: 4. Threshold = 0.50. MAE = 23.93, 5 rul

In [5]:
for i, (rem, feat, target, algorithm, model) in enumerate(zip(toRemove, features, targets, algorithms, models)):

    print(algorithm, target, len(feat))

    if i != 4:
        print("skip")
        continue

    name = f"test/resources/datasets/datasets/{algorithm}.csv"

    if not exists(name):
        process(algorithm)

    dataset = pd.read_csv(name)

    dataset = dataset.drop(rem, axis = 1)

    train, test = train_test_split(dataset, test_size=0.2, random_state=10)
    model.fit(train.iloc[:, :-1], train.iloc[:, -1])
    E = abs(model.predict(test.iloc[:, :-1]) - test.iloc[:, -1])
    print("MAE = {:.2f}".format(E.mean()))
    output.append(("model", algorithm, target, E.mean()))

    #crash = CRASH(model, train, readability_tradeoff=0.5, max_depth=5, patience=2,
    #              algorithm=CRASH.Algorithm.CREAM, objective=Objective.MODEL)
    #crash.search()
    #best =  crash.get_best()[0]

    #cream = Extractor.creepy(model, depth=best[2], error_threshold=best[3], constant=False)
    #theory_from_cream = cream.extract(train)
    #print('CReEPy performance ({} rules):\nMAE = {:.2f}\nMAE fidelity = {:.2f}\n'
    #      .format(cream.n_rules, cream.mae(test), cream.mae(test, model)))

    #print()

    model = KNeighborsRegressor(n_neighbors=1).fit(train.iloc[:, :-1], train.iloc[:, -1])
    crash = CRASH(model, train, readability_tradeoff=0.5, max_depth=5, patience=2,
                  algorithm=CRASH.Algorithm.CREAM, objective=Objective.DATA)
    crash.search()
    best =  crash.get_best()[0]

    cream = Extractor.creepy(model, depth=best[2], error_threshold=best[3], constant=False)
    theory_from_cream = cream.extract(train)
    print('CReEPy performance ({} rules):\nMAE = {:.2f}\nMAE fidelity = {:.2f}\n'
          .format(cream.n_rules, cream.mae(test), cream.mae(test, model)))

    print()

contingency memo 1
skip
contingency time 1
skip
anticipate memo 1
skip
anticipate time 1
skip
contingency cost 5
MAE = 1.50
Algorithm.CREAM. Depth: 1. Threshold = 0.00. MAE = 29.47, 2 rules
Algorithm.CREAM. Depth: 1. Threshold = 0.00. MAE = 29.37, 2 rules
Algorithm.CREAM. Depth: 1. Threshold = 0.00. MAE = 29.40, 2 rules
Algorithm.CREAM. Depth: 1. Threshold = 29.40. MAE = 29.27, 2 rules
Algorithm.CREAM. Depth: 1. Threshold = 58.80. MAE = 29.31, 2 rules

Algorithm.CREAM. Depth: 2. Threshold = 0.00. MAE = 29.21, 4 rules
Algorithm.CREAM. Depth: 2. Threshold = 0.00. MAE = 29.14, 4 rules
Algorithm.CREAM. Depth: 2. Threshold = 0.00. MAE = 29.21, 4 rules
Algorithm.CREAM. Depth: 2. Threshold = 29.21. MAE = 28.99, 3 rules
Algorithm.CREAM. Depth: 2. Threshold = 58.41. MAE = 29.37, 2 rules
Algorithm.CREAM. Depth: 2. Threshold = 87.62. MAE = 29.37, 2 rules
Algorithm.CREAM. Depth: 2. Threshold = 116.83. MAE = 29.31, 2 rules
Algorithm.CREAM. Depth: 2. Threshold = 146.04. MAE = 29.47, 2 rules

Algorit

In [25]:
%timeit -r 100 -n 1 print(5, end="\r")

The slowest run took 113.80 times longer than the fastest. This could mean that an intermediate result is being cached.
50 µs ± 137 µs per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [103]:
import time

i = 0

rem, feat, target, algorithm, model = toRemove[i], features[i], targets[i], algorithms[i], models[i]

print(algorithm, target, len(feat), rem[:-2], rem[-2:])
print()
name = f"test/resources/datasets/datasets/{algorithm}.csv"

if not exists(name):
    process(algorithm)

dataset = pd.read_csv(name).drop(rem[-2 :], axis = 1)

m = {}
e = {}

for r in range(-1, len(rem) - 2):
    if r >= 0:
        dataset = dataset.drop([rem[r]], axis = 1)

    train, test = train_test_split(dataset, test_size=0.1, random_state=10)
    model.fit(train.iloc[:, :-1], train.iloc[:, -1])

    print(f"{len(dataset.columns) - 1} variables\n")
    dat = []
    for j in [100, 500, 1000, 2000, 4000, 7000, 10000]:
        print(j, "instances")
        res = []
        for i in range(100):
            print(i, end="\r")
            t0 = time.time()
            creepy = Extractor.creepy(model, depth=1, error_threshold=.8, constant=False)
            theory_from_creepy = creepy.extract(train.iloc[:j, :])
            t1 = time.time()
            res.append(t1 - t0)
        res = np.array(res)
        print(f'{np.mean(res):.2f} +- {np.std(res):.2f}')
        print()
        dat.append((np.mean(res), np.std(res)))
    m[len(dataset.columns) - 1] = [d[0] for d in dat]
    e[len(dataset.columns) - 1] = [d[1] for d in dat]
    print()
print(m)
print(e)

contingency memo 1 ['PV_mean', 'PV_std', 'Load_mean', 'Load_std'] ['time', 'cost']

5 variables

100 instances
0.03 +- 0.01

500 instances
0.04 +- 0.01

1000 instances
0.08 +- 0.02

2000 instances
0.24 +- 0.04

4000 instances
0.40 +- 0.07

7000 instances
1.49 +- 0.16

10000 instances
3.24 +- 0.48


4 variables

100 instances
0.03 +- 0.01

500 instances
0.04 +- 0.01

1000 instances
0.08 +- 0.01

2000 instances
0.19 +- 0.02

4000 instances
0.44 +- 0.10

7000 instances
1.18 +- 0.06

10000 instances
3.27 +- 0.34


3 variables

100 instances
0.03 +- 0.01

500 instances
0.05 +- 0.01

1000 instances
0.06 +- 0.01

2000 instances
0.15 +- 0.01

4000 instances
0.35 +- 0.03

7000 instances
0.96 +- 0.05

10000 instances
2.01 +- 0.18


2 variables

100 instances
0.03 +- 0.00

500 instances
0.04 +- 0.01

1000 instances
0.07 +- 0.01

2000 instances
0.12 +- 0.02

4000 instances
0.32 +- 0.07

7000 instances
0.66 +- 0.04

10000 instances
2.09 +- 0.24


1 variables

100 instances
0.05 +- 0.01

500 instanc

In [104]:
import time

i = 0

rem, feat, target, algorithm, model = toRemove[i], features[i], targets[i], algorithms[i], models[i]

print(algorithm, target, len(feat), rem[:-2], rem[-2:])
print()
name = f"test/resources/datasets/datasets/{algorithm}.csv"

if not exists(name):
    process(algorithm)

dataset = pd.read_csv(name).drop(rem[-2 :], axis = 1)

m = {}
e = {}

for r in range(-1, len(rem) - 2):
    if r >= 0:
        dataset = dataset.drop([rem[r]], axis = 1)

    train, test = train_test_split(dataset, test_size=0.1, random_state=10)
    model.fit(train.iloc[:, :-1], train.iloc[:, -1])

    print(f"{len(dataset.columns) - 1} variables\n")
    dat = []
    for j in [100, 500, 1000, 2000, 4000, 7000, 10000]:
        print(j, "instances")
        res = []
        for i in range(100):
            print(i, end="\r")
            t0 = time.time()
            cream = Extractor.cream(model, depth=1, error_threshold=.8, constant=False)
            theory_from_cream = cream.extract(train.iloc[:j, :])
            t1 = time.time()
            res.append(t1 - t0)
        res = np.array(res)
        print(f'{np.mean(res):.2f} +- {np.std(res):.2f}')
        print()
        dat.append((np.mean(res), np.std(res)))
    m[len(dataset.columns) - 1] = [d[0] for d in dat]
    e[len(dataset.columns) - 1] = [d[1] for d in dat]
    print()
print(m)
print(e)

contingency memo 1 ['PV_mean', 'PV_std', 'Load_mean', 'Load_std'] ['time', 'cost']

5 variables

100 instances
0.04 +- 0.01

500 instances
0.05 +- 0.01

1000 instances
0.11 +- 0.02

2000 instances
0.25 +- 0.04

4000 instances
0.43 +- 0.09

7000 instances
1.53 +- 0.22

10000 instances
3.14 +- 0.44


4 variables

100 instances
0.04 +- 0.01

500 instances
0.05 +- 0.01

1000 instances
0.08 +- 0.00

2000 instances
0.19 +- 0.02

4000 instances
0.40 +- 0.01

7000 instances
1.21 +- 0.10

10000 instances
3.15 +- 0.21


3 variables

100 instances
0.04 +- 0.01

500 instances
0.05 +- 0.01

1000 instances
0.07 +- 0.01

2000 instances
0.16 +- 0.01

4000 instances
0.35 +- 0.02

7000 instances
0.97 +- 0.08

10000 instances
1.98 +- 0.17


2 variables

100 instances
0.04 +- 0.01

500 instances
0.06 +- 0.01

1000 instances
0.08 +- 0.01

2000 instances
0.11 +- 0.01

4000 instances
0.33 +- 0.07

7000 instances
0.64 +- 0.03

10000 instances
2.01 +- 0.12


1 variables

100 instances
0.05 +- 0.01

500 instanc